### Experiement with how to reduct the docuemnt to be parsed. Number of tokens is too hight. Try to experiement with date.

In [148]:
import openai
import docx
import pandas as pd
FILE_NAME = "/Users/prashantgaurav/Projects/KaajAi/receipts/bank-statement-template-09.docx"

def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

In [149]:
import openai
import os
client = openai.Client(api_key=os.environ.get("OPENAI_API_KEY"))
text = getText(FILE_NAME)

In [150]:
doc = docx.Document(FILE_NAME)
text = ""
for table in doc.tables:
    for row in table.rows:
        for cell in row.cells:
            text+= cell.text

In [151]:
print(text)

DateDetailsWithdrawalsDepositsBalanceApr 8Opening Balance5,234.09Apr 8Insurance272.455,506.54Apr 10ATM200.005,306.54Apr 12Internet Transfer250.005,556.54Apr 12Payroll2100.007,656.54Apr 13Bill payment135.077,521.47Apr 14Direct debit200.007,321.47Apr 14Deposit250.007.567.87Apr 15Bill payment525.727,042.15Apr 17Bill payment327.636,714.52Apr 17Bill payment729.965,984.56Apr 18Bill payment223.695,710.87Closing Balance$5,710.87


In [152]:
%%time
prompt = """I'm going to give you the contents of a pdf of a bank statement. The text containes transaction records along with other things.
you job is to parse each transaction record and return the details like date, description, amount and type. The type column should contain either 
'credit' or 'debit'. Return these information in a tabular format. Do not return anything else just the table"""

docs = [{"role": "system", "content": prompt + '\n' + text}]
response = client.chat.completions.create(
                model="gpt-4",
                messages=docs
            )

resp = response.choices[0].message.content

CPU times: user 17.3 ms, sys: 9.72 ms, total: 27.1 ms
Wall time: 8.4 s


In [153]:
print(resp)

| Date | Description | Amount | Type |
|------|-------------|--------|------|
| Apr 8 | Insurance | 272.45 | Debit |
| Apr 10 | ATM | 200.00 | Debit |
| Apr 12 | Internet Transfer | 250.00 | Debit |
| Apr 12 | Payroll | 2100.00 | Credit |
| Apr 13 | Bill payment | 135.07 | Debit |
| Apr 14 | Direct debit | 200.00 | Debit |
| Apr 14 | Deposit | 250.00 | Credit |
| Apr 15 | Bill payment | 525.72 | Debit |
| Apr 17 | Bill payment | 327.63 | Debit |
| Apr 17 | Bill payment | 729.96 | Debit |
| Apr 18 | Bill payment | 223.69 | Debit |


In [154]:
def parse_response_table(resp):
    lines = resp.strip().split("\n")
    table = [line.split('|') for line in lines]
    final_table = []
    for line in table:
        new_line = []
        for ele in line:
            ele = ele.replace('-','')
            if ele :
               new_line.append(ele.strip())
        if not new_line:
            continue
        final_table.append(new_line)
    print(final_table)
    df = pd.DataFrame(final_table[1:],columns=final_table[0])
    return df
df = parse_response_table(resp)

[['Date', 'Description', 'Amount', 'Type'], ['Apr 8', 'Insurance', '272.45', 'Debit'], ['Apr 10', 'ATM', '200.00', 'Debit'], ['Apr 12', 'Internet Transfer', '250.00', 'Debit'], ['Apr 12', 'Payroll', '2100.00', 'Credit'], ['Apr 13', 'Bill payment', '135.07', 'Debit'], ['Apr 14', 'Direct debit', '200.00', 'Debit'], ['Apr 14', 'Deposit', '250.00', 'Credit'], ['Apr 15', 'Bill payment', '525.72', 'Debit'], ['Apr 17', 'Bill payment', '327.63', 'Debit'], ['Apr 17', 'Bill payment', '729.96', 'Debit'], ['Apr 18', 'Bill payment', '223.69', 'Debit']]


In [155]:
df

,Date,Description,Amount,Type
0,Apr 8,Insurance,272.45,Debit
1,Apr 10,ATM,200.00,Debit
2,Apr 12,Internet Transfer,250.00,Debit
3,Apr 12,Payroll,2100.00,Credit
4,Apr 13,Bill payment,135.07,Debit
5,Apr 14,Direct debit,200.00,Debit
6,Apr 14,Deposit,250.00,Credit
7,Apr 15,Bill payment,525.72,Debit
8,Apr 17,Bill payment,327.63,Debit
9,Apr 17,Bill payment,729.96,Debit
